In [ ]:
import pandas as pd

co2_data = pd.read_csv('owid-co2-data.csv')
temperature_data = pd.read_csv('GLB.Ts+dSST.csv')
natural_disasters_data = pd.read_csv('number-of-natural-disaster-events.csv')
gdp_growth_data = pd.read_csv('real-gdp-growth.csv')


In [ ]:
print(co2_data.columns)
print(temperature_data.columns)
print(natural_disasters_data.columns)
print(gdp_growth_data.columns)

Index(['country', 'year', 'iso_code', 'population', 'gdp', 'cement_co2',
       'cement_co2_per_capita', 'co2', 'co2_growth_abs', 'co2_growth_prct',
       'co2_including_luc', 'co2_including_luc_growth_abs',
       'co2_including_luc_growth_prct', 'co2_including_luc_per_capita',
       'co2_including_luc_per_gdp', 'co2_including_luc_per_unit_energy',
       'co2_per_capita', 'co2_per_gdp', 'co2_per_unit_energy', 'coal_co2',
       'coal_co2_per_capita', 'consumption_co2', 'consumption_co2_per_capita',
       'consumption_co2_per_gdp', 'cumulative_cement_co2', 'cumulative_co2',
       'cumulative_co2_including_luc', 'cumulative_coal_co2',
       'cumulative_flaring_co2', 'cumulative_gas_co2', 'cumulative_luc_co2',
       'cumulative_oil_co2', 'cumulative_other_co2', 'energy_per_capita',
       'energy_per_gdp', 'flaring_co2', 'flaring_co2_per_capita', 'gas_co2',
       'gas_co2_per_capita', 'ghg_excluding_lucf_per_capita', 'ghg_per_capita',
       'land_use_change_co2', 'land_use_chang

In [7]:
import pandas as pd
from google.colab import files


co2_df = pd.read_csv('owid-co2-data.csv')
temp_df = pd.read_csv('GLB.Ts+dSST.csv', usecols=lambda column: column not in ['Land-Ocean: Global Means'])
disasters_df = pd.read_csv('number-of-natural-disaster-events.csv')
gdp_df = pd.read_csv('real-gdp-growth.csv')



gdp_df.rename(columns={'Entity': 'country', 'Year': 'year'}, inplace=True)
disasters_df.rename(columns={'Entity': 'country', 'Year': 'year'}, inplace=True)
temp_df.rename(columns={'Year': 'year'}, inplace=True)

print(temp_df.columns)
merged_df = pd.merge(gdp_df, co2_df, on=['country', 'year'], how='outer')
merged_df = pd.merge(merged_df, disasters_df, on=['country', 'year'], how='outer')

merged_df.fillna(merged_df.mean(), inplace=True)

print(merged_df.head())
merged_df.to_csv('merged_dataset.csv', index=False)
files.download('merged_dataset.csv')

Index([], dtype='object')


<ipython-input-7-c6ee762fa1ee>:30: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  merged_df.fillna(merged_df.mean(), inplace=True)


       country Code_x  year  \
0  Afghanistan    AFG  2003   
1  Afghanistan    AFG  2004   
2  Afghanistan    AFG  2005   
3  Afghanistan    AFG  2006   
4  Afghanistan    AFG  2007   

   Gross domestic product, constant prices - Percent change iso_code  \
0                                              8.692             AFG   
1                                              0.671             AFG   
2                                             11.830             AFG   
3                                              5.361             AFG   
4                                             13.340             AFG   

   population           gdp  cement_co2  cement_co2_per_capita    co2  ...  \
0  22645136.0  2.107434e+10       0.010                    0.0  1.560  ...   
1  23553554.0  2.233257e+10       0.010                    0.0  1.237  ...   
2  24411196.0  2.539769e+10       0.006                    0.0  1.890  ...   
3  25442946.0  2.870440e+10       0.012                    0.0  2.15

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
!pip install pandas statsmodels

In [16]:
import pandas as pd
import statsmodels.api as sm
from google.colab import files

df = pd.read_csv('merged_dataset.csv')

X = df[['Gross domestic product, constant prices - Percent change', 'population','energy_per_gdp']]
X = sm.add_constant(X)

y = df['total_ghg']

model = sm.OLS(y, X).fit()
residuals = model.resid
df['Residuals'] = residuals

print(df['Residuals'].head())

print(model.summary())


0   -611.508278
1   -624.832976
2   -612.064527
3   -624.170283
4   -613.783629
Name: Residuals, dtype: float64
                            OLS Regression Results                            
Dep. Variable:              total_ghg   R-squared:                       0.506
Model:                            OLS   Adj. R-squared:                  0.506
Method:                 Least Squares   F-statistic:                 1.806e+04
Date:                Mon, 27 Nov 2023   Prob (F-statistic):               0.00
Time:                        18:51:07   Log-Likelihood:            -4.3347e+05
No. Observations:               52873   AIC:                         8.669e+05
Df Residuals:                   52869   BIC:                         8.670e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                                               coef    std err          t      P>|

<ipython-input-16-6cf137d0014a>:10: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('merged_dataset.csv')


In [17]:
sum_of_residuals = df['Residuals'].sum()

print("Sum of Residuals:", sum_of_residuals)

Sum of Residuals: 0.012347820797003806


In [19]:
print(len(residuals))

52873


In [16]:
import pandas as pd
import statsmodels.api as sm

df = pd.read_csv('merged_dataset.csv')

df['Temp_t-1'] = df['temperature_change_from_co2'].shift(1)

df['Temp_t-1'].fillna(method='bfill', inplace=True)


df['total_ghg'].fillna(df['total_ghg'].mean(), inplace=True)

X = df[['total_ghg', 'Temp_t-1']]
X = sm.add_constant(X)

y = df['temperature_change_from_ghg']

model = sm.OLS(y, X, missing='drop').fit()

print(model.summary())


                                 OLS Regression Results                                
Dep. Variable:     temperature_change_from_ghg   R-squared:                       0.960
Model:                                     OLS   Adj. R-squared:                  0.960
Method:                          Least Squares   F-statistic:                 6.282e+05
Date:                         Tue, 28 Nov 2023   Prob (F-statistic):               0.00
Time:                                 06:34:23   Log-Likelihood:             1.6738e+05
No. Observations:                        52873   AIC:                        -3.347e+05
Df Residuals:                            52870   BIC:                        -3.347e+05
Df Model:                                    2                                         
Covariance Type:                     nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

<ipython-input-16-d641ff4d3b02>:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('merged_dataset.csv')


In [20]:
import pandas as pd
import statsmodels.api as sm


data = pd.read_csv('merged_dataset.csv')


X = data['temperature_change_from_co2']
y = data['land_use_change_co2_per_capita']

X = sm.add_constant(X)

X['Temp_squared'] = X['temperature_change_from_co2'] ** 2

model = sm.OLS(y, X).fit()

print(model.params)


const                           4.186534
temperature_change_from_co2   -15.170869
Temp_squared                   13.428534
dtype: float64


<ipython-input-20-32ce7ef0fe44>:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('merged_dataset.csv')


In [ ]:
import pandas as pd
import statsmodels.api as sm


data = pd.read_csv('merged_dataset.csv')

E_t = data['total_ghg']
E_target =  # Defining the emissions target level here
C_Mt = data['MitigationCost']

E_diff = E_t - E_target

E_diff = sm.add_constant(E_diff)

model = sm.OLS(C_Mt, E_diff).fit()

print('Estimated theta value:', model.params[1])

